In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['sample_submission.csv', 'test.json', 'train.json']


In [2]:
import numpy as np
import pandas as pd

In [3]:
import json
from pprint import pprint

with open('../input/train.json') as f:
    data = json.load(f)

In [4]:
_id = list()
cuisine = list()
ingre = list()

for i in range(len(data)):
    _id.append(data[i]['id'])
    cuisine.append(data[i]['cuisine'])
    ingre.append(data[i]['ingredients'])

In [5]:
all_in = list()

for i in ingre:
    for j in i:
        all_in.append(j)

In [6]:
vocab = set(all_in)

In [7]:
word_to_index = dict()
index_to_word = dict()
index = 1

for i in vocab:
    word_to_index[i] = index
    index_to_word[index] = i
    index += 1

In [8]:
seq = list()

for i in ingre:
    list_ = list()
    for j in i:
        x = word_to_index.get(j,-1)
        if x != -1:
            list_.append(x)
    seq.append(list_)

In [9]:
m = 0

for i in range(len(seq)):
    if m < len(seq[i]):
        m = len(seq[i])

In [10]:
m

65

In [11]:
from keras.preprocessing.sequence import pad_sequences
seq = pad_sequences(seq, maxlen=65)

Using TensorFlow backend.


In [12]:
seq = np.array(seq)

In [13]:
cuisine = pd.get_dummies(cuisine)

In [14]:
cuisine.shape

(39774, 20)

In [15]:
with open('../input/test.json') as f:
    test_data = json.load(f)

In [16]:
test_id = list()
# test_cuisine = list()
test_ingre = list()

for i in range(len(test_data)):
    test_id.append(test_data[i]['id'])
#     test_cuisine.append(test_data[i]['cuisine'])
    test_ingre.append(test_data[i]['ingredients'])

In [17]:
test_seq = list()

for i in test_ingre:
    list_ = list()
    for j in i:
        x = word_to_index.get(j,-1)
        if x != -1:
            list_.append(x)
    test_seq.append(list_)

In [18]:
test_seq = pad_sequences(test_seq, maxlen=65)

In [20]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
embedding_size=64
model=Sequential()
model.add(Embedding(len(vocab)+1, embedding_size, input_length=65))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128))
model.add(Dense(20, activation='softmax'))
print(model.summary())

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 65, 64)            429760    
_________________________________________________________________
lstm_1 (LSTM)                (None, 65, 128)           98816     
_________________________________________________________________
dropout_1 (Dropout)          (None, 65, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 20)                2580      
Total params: 662,740
Trainable params: 662,740
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
from keras.callbacks import ModelCheckpoint

# checkpoint
filepath="weights-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]


model.compile(loss='categorical_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

In [23]:
model.fit(seq, cuisine, batch_size=64, epochs=15)

Epoch 1/15
39774/39774 [==============================] - 111s 3ms/step - loss: 0.5262 - acc: 0.8448
Epoch 2/15
39774/39774 [==============================] - 110s 3ms/step - loss: 0.4731 - acc: 0.8595
Epoch 3/15
39774/39774 [==============================] - 111s 3ms/step - loss: 0.4446 - acc: 0.8665
Epoch 4/15
39774/39774 [==============================] - 111s 3ms/step - loss: 0.4134 - acc: 0.8750
Epoch 5/15
39774/39774 [==============================] - 110s 3ms/step - loss: 0.3872 - acc: 0.8825
Epoch 6/15
39774/39774 [==============================] - 112s 3ms/step - loss: 0.3607 - acc: 0.8910
Epoch 7/15
39774/39774 [==============================] - 111s 3ms/step - loss: 0.3357 - acc: 0.8992
Epoch 8/15
39774/39774 [==============================] - 111s 3ms/step - loss: 0.3109 - acc: 0.9047
Epoch 9/15
39774/39774 [==============================] - 112s 3ms/step - loss: 0.2941 - acc: 0.9090
Epoch 10/15
39774/39774 [==============================] - 111s 3ms/step - loss: 0.2757 - a

In [24]:
pred = model.predict(seq[1].reshape(-1,65))

In [25]:
np.argmax(pred)

16

In [26]:
cuisine.head()

,brazilian,british,cajun_creole,chinese,filipino,french,greek,indian,irish,italian,jamaican,japanese,korean,mexican,moroccan,russian,southern_us,spanish,thai,vietnamese
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
names = list(cuisine.keys())
names[np.argmax(pred)]

'southern_us'

In [28]:
data[1]

{'id': 25693,
 'cuisine': 'southern_us',
 'ingredients': ['plain flour',
  'ground pepper',
  'salt',
  'tomatoes',
  'ground black pepper',
  'thyme',
  'eggs',
  'green tomatoes',
  'yellow corn meal',
  'milk',
  'vegetable oil']}

In [29]:
test_pred = list()

for i in test_seq:
    pred = model.predict(i.reshape(-1,65))
    test_pred.append(names[np.argmax(pred)])

In [30]:
test_pred = np.array(test_pred).reshape(-1,1)
test_id = np.array(test_id).reshape(-1,1)

In [31]:
output = np.array(np.concatenate((test_id, test_pred),axis=1))

In [32]:
output[0]

array(['18009', 'british'], dtype='<U21')

In [33]:
output = pd.DataFrame(output,columns = ["id","cuisine"])

In [34]:
output.to_csv('out.csv',index = False)